In [1]:

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import os
import pandas as pd
from time import time
import matplotlib.pyplot as plt
# from binarized_modules import  BinarizeLinear,BinarizeConv2d

In [2]:
import os
os.chdir("/home/earapidis/Desktop/BinaryNet.pytorch")
os.getcwd()

'/home/earapidis/Desktop/BinaryNet.pytorch'

In [3]:
import models.binarized_modules

In [4]:
import importlib
importlib.reload(models.binarized_modules)

<module 'models.binarized_modules' from '/home/earapidis/Desktop/BinaryNet.pytorch/models/binarized_modules.py'>

In [5]:
cuda = True

In [6]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

In [7]:
test_batch_size=1000

In [8]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [9]:
import models.lenet_5


importlib.reload(models.lenet_5)

<module 'models.lenet_5' from '/home/earapidis/Desktop/BinaryNet.pytorch/models/lenet_5.py'>

In [10]:
# from models.lenet_5 import BinarizedLeNet5_NoBN as Net
from models.lenet_5 import BinarizedLeNet5_BN as Net

In [11]:
def test(model ):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            if cuda:
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    acc = 100. * correct / len(test_loader.dataset)
    return acc

In [12]:
model_sim = 2

save_dir = os.path.abspath("/shares/bulk/earapidis/saved_models/lenet_5")
models_path = os.path.join(save_dir,f"model_{model_sim}")

In [13]:
# model_path = os.path.join(models_path,f"epoch_7.pth")
model_path = os.path.join(models_path,f"best.pth")
model = Net()
model.load_state_dict(torch.load(model_path))
if cuda:
    torch.cuda.set_device(0)
    model.cuda()

In [17]:
lr=0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


In [18]:
model.state_dict

<bound method Module.state_dict of BinarizedLeNet5_BN(
  (conv1): BinarizeConv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh1): Hardtanh(min_val=-1.0, max_val=1.0)
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): BinarizeConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh2): Hardtanh(min_val=-1.0, max_val=1.0)
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): BinarizeLinear(in_features=256, out_features=120, bias=True)
  (bn_fc1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh3): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc2): BinarizeLinear(in_features=120, out_features=84, bias=True)
  (bn_fc2): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh4): Hardtanh(min_val=-1.0, max_val=1.0)
  (

In [19]:
test(model)


Test set: Average loss: 0.0001, Accuracy: 9712/10000 (97%)



tensor(97.1200)

In [24]:
from models.binarized_modules import  BinarizeLinear,BinarizeConv2d, binarized


In [29]:
weights = {
    name: binarized(p.detach().cpu().clone())       # binarize weights only
    for name, p in model.named_parameters()
    if 'weight' in name and not any(skip in name for skip in ['bn', 'bias'])
}


In [30]:
weights.keys()

dict_keys(['conv1.weight', 'conv2.weight', 'fc1.weight', 'fc2.weight', 'fc3.weight'])

In [54]:
a = weights["conv2.weight"]
print(a.shape)
a

torch.Size([16, 6, 5, 5])


tensor([[[[-1.,  1., -1., -1., -1.],
          [-1., -1.,  1., -1., -1.],
          [-1., -1., -1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.,  1.],
          [ 1.,  1.,  1., -1.,  1.]],

         [[-1., -1.,  1.,  1., -1.],
          [-1., -1., -1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.,  1.],
          [ 1.,  1.,  1., -1.,  1.],
          [-1., -1., -1.,  1.,  1.]],

         [[ 1., -1., -1., -1., -1.],
          [-1., -1., -1.,  1.,  1.],
          [-1.,  1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.,  1.],
          [-1.,  1., -1.,  1.,  1.]],

         [[ 1., -1., -1., -1.,  1.],
          [ 1.,  1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.,  1.],
          [-1., -1., -1.,  1.,  1.],
          [-1.,  1.,  1.,  1., -1.]],

         [[ 1.,  1.,  1.,  1.,  1.],
          [-1., -1., -1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.,  1.],
          [ 1.,  1.,  1., -1.,  1.]],

         [[ 1.,  1.,  1., -1.,  1.],
          [ 1.,  1., -1., -1

In [40]:
activations = {}

def save_activation_binarized_selectively(name, binarize=True):
    def hook(module, inp, out):
        act = out.detach().cpu()
        if binarize:
            act = binarized(act)
        activations[name] = act
    return hook


In [41]:
for name, module in model.named_modules():
    if isinstance(module, (BinarizeConv2d, BinarizeLinear)):
        module.register_forward_hook(save_activation_binarized_selectively(name, binarize=True))
    elif isinstance(module, (nn.BatchNorm2d, nn.BatchNorm1d, nn.AvgPool2d, nn.MaxPool2d)):
        module.register_forward_hook(save_activation_binarized_selectively(name, binarize=False))


In [43]:
model.eval()
with torch.no_grad():
    x, _ = next(iter(test_loader))
    if cuda:
        x = x.cuda()
    _ = model(x)


In [46]:
activations.keys()

dict_keys(['conv1', 'bn1', 'pool1', 'conv2', 'bn2', 'pool2', 'fc1', 'bn_fc1', 'fc2', 'bn_fc2', 'fc3', 'bn_fc3'])

In [48]:
activations["conv2"]

tensor([[[[ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
          [-1., -1., -1.,  ...,  1.,  1.,  1.],
          [-1.,  1.,  1.,  ...,  1.,  1.,  1.],
          ...,
          [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
          [ 1., -1., -1.,  ..., -1., -1.,  1.]],

         [[-1., -1.,  1.,  ...,  1., -1., -1.],
          [ 1.,  1.,  1.,  ...,  1.,  1., -1.],
          [ 1.,  1., -1.,  ...,  1., -1.,  1.],
          ...,
          [-1., -1.,  1.,  ..., -1.,  1.,  1.],
          [-1., -1., -1.,  ...,  1.,  1., -1.],
          [-1.,  1.,  1.,  ...,  1.,  1., -1.]],

         [[-1.,  1.,  1.,  ..., -1., -1., -1.],
          [ 1.,  1.,  1.,  ..., -1., -1., -1.],
          [ 1., -1., -1.,  ...,  1., -1., -1.],
          ...,
          [ 1.,  1.,  1.,  ..., -1., -1.,  1.],
          [-1.,  1.,  1.,  ...,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  ...,  1.,  1., -1.]],

         ...,

         [[-1., -1., -1.,  ...,  1., -1., -1.],
          [-1., -1.,  